## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделим выборку на обучающую/проверочную в соотношении 80/20.

Применим логистическую регрессию по всему набору данных.

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

### Загрузка данных

In [2]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

In [3]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)
for l in data["Product_Info_2_1"].unique():
    data["Product_Info_2_1" + l] = data["Product_Info_2_1"].isin([l]).astype("int8")
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data.fillna(value=-1, inplace=True)

### Оптимизация памяти

In [4]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [5]:
data = reduce_mem_usage(data)
print (data.info())

Потребление памяти меньше на 49.49 Мб (минус 84.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 133 entries, Id to Product_Info_2_1B
dtypes: float16(18), int16(1), int32(1), int8(113)
memory usage: 8.8 MB
None


### Набор столбцов для расчета

In [6]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Предобработка данных
Дополнительно проведем z-нормализацию данных через предварительную обработку (preprocessing).

In [7]:
scaler = preprocessing.StandardScaler()
scaler.fit(pd.DataFrame(data, columns=columns))

StandardScaler()

### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [8]:
data_train, data_test = train_test_split(data, test_size=0.2)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

          Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
7333    9778               1              26        0.076904               2   
35161  46703               1              26        0.051270               2   
22004  29305               1              26        0.076904               2   
1950    2629               1              26        0.230713               2   
39704  52724               2              26        0.179443               2   

       Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
7333                3               1  0.507324  0.745605  0.299072  ...   
35161               3               1  0.298584  0.654785  0.288818  ...   
22004               3               1  0.775879  0.636230  0.225952  ...   
1950                3               3  0.268555  0.672852  0.177856  ...   
39704               3               1  0.044769  0.600098  0.196655  ...   

       Medical_Keyword_46  Medical_Keyword_47  Medical_Keyword

### Логистическая регрессия
\begin{equation}
P = \frac{exp ^{\ T}}{1+exp ^{\ T}}
\end{equation}
\begin{equation}
T = a_0 + b_1x_1 + \cdots + b_nx_n
\end{equation}
T - терминатор, логистическая кривая

![](https://scikit-learn.org/0.22/_images/sphx_glr_plot_sgd_iris_001.png "")

In [9]:
def regression_model (df, columns):
    y = df["Response"]
    x = scaler.transform(pd.DataFrame(df, columns=columns))
    model = LogisticRegression(max_iter=1000, class_weight='balanced',
                              multi_class='multinomial')
    model.fit(x, y)
    return model

def logistic_regression (columns):
    x = scaler.transform(pd.DataFrame(data_test, columns=columns))
    model = regression_model(data_train, columns)
    data_test["target"] = model.predict(x)
    return cohen_kappa_score(data_test["target"],
                        data_test["Response"], weights="quadratic")

### Предсказание данных и оценка модели
Кластеризация дает 0.192, kNN(100) - 0.3

In [10]:
print ("Логистическая регрессия:",
       round(logistic_regression(columns), 3))

Логистическая регрессия: 0.501


В соревновании на Kaggle 0.512 - **2248 место**

### Матрица неточностей

In [11]:
print (confusion_matrix(data_test["target"], data_test["Response"]))

[[ 337  206    5   10   77  194  104  102]
 [ 210  301    7    2  135  183   68   50]
 [ 101  151  102   59  116  250   41   29]
 [  76   60   60  198   28  398   73  117]
 [ 140  247   11    3  463  296  164  160]
 [  53   59    7   10   56  258  119  150]
 [ 149  142    0    4  112  352  637  404]
 [ 172  167   10   18  126  299  385 2854]]
